In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/information_from_links')

Now that we have the data, we get to the fun task of cleaning up our data and making it ready for modelling.

The flow will be as follows:

1. Check for null entries and deal with them
2. Clean columns one by one (remove \n and \t) and convert all entries to lower case. Do not split any columns here.
3. Convert dates into datetime objects
4. Convert compensation into categories
5. Work with skills and job loc
6. Replace some repetitive keywords in details by space
7. Remove any duplicate entries

Then we will decide on what features to consider for our knowledge based part and what all to consider for our content based part of the recommendation sysytem.

In [3]:
df.head(1)

,Unnamed: 0,href,job_title,company_name,job_loc,details,category,compensation,start,end,skills
0,0,http://letsintern.com/internship/Human-Resourc...,\nHR Executive - Recruitment\n,\nEngenia Technologies\n,\n\nGurgaon\n\n,\nAbout Internship:\n\t\t\t\t\t\t\t\t\t\t\t\t\...,Human Resources Recruiter,Recurring,2nd Mar 2019,28th Aug 2019,\nSkills Required\n\n\n\t\t\t\t\t\t\t\t\t\t\t\...


In [4]:
del df['Unnamed: 0']

1. NULL ENTRIES

In [5]:
df.isnull().sum()

href            0
job_title       0
company_name    0
job_loc         0
details         0
category        0
compensation    0
start           0
end             0
skills          0
dtype: int64

Phew ! Dodged a bullet there

2. REMOVING \t AND \n AND LOWER CASE TRANSFORM

In [6]:
df = df.apply(lambda x: x.str.replace('\t',' ')).apply(lambda x: x.str.replace('\n',' '))

We also discovered later that \r was creating a lot of problem and thus we should be removing that too

In [7]:
df = df.apply(lambda x: x.str.replace('\r',' '))

In [8]:
# removing any extra space created because of the above steps
df = df.apply(lambda x: x.str.strip())

In [9]:
df.head()

,href,job_title,company_name,job_loc,details,category,compensation,start,end,skills
0,http://letsintern.com/internship/Human-Resourc...,HR Executive - Recruitment,Engenia Technologies,Gurgaon,About Internship: We are seeking...,Human Resources Recruiter,Recurring,2nd Mar 2019,28th Aug 2019,Skills Required HR Practices
1,http://letsintern.com/internship/Tele-Sales-Ex...,Telecalling & Lead Generation,Abalone Technologies Pvt Ltd,Noida,About Internship: Selected inter...,Tele Sales Executive,Recurring,17th Feb 2019,30th Aug 2019,Skills Required Office Admini...
2,http://letsintern.com/internship/Marketing-Pro...,Digital Marketing Internship,Brandstory Digital Marketing Company,Bangalore,About Internship: Are you lookin...,Marketing Professional,Recurring,25th Dec 2018,29th Apr 2020,Skills Required Digital Marke...
3,http://letsintern.com/internship/Accountant-in...,RECRUITMENT OF CORPORATE BANK BACK OFFICE POST,bandhan pvt.ltd,"Kathua, Barasat, ...",About Internship: Huge Opportuni...,Accountant,Expenses Covered,12th Mar 2019,Nov ---- Nov -,Skills Required Analytical Sk...
4,http://letsintern.com/internship/Software-Deve...,Software Developer,TrippyIgloo,Bangalore,About Internship: We are looking...,Software Developer : Python,Recurring,30th Jan 2019,20th Jun 2019,Skills Required GO(Golang) ...


Lets take a look at some cases that we think might still have problems:


In [10]:
df.job_loc.value_counts()

Anywhere in India                                                                                                                                                                                                                                                                                                                                               93
Bangalore                                                                                                                                                                                                                                                                                                                                                       79
Mumbai                                                                                                                                                                                                                                                                                            

Keep in mind that different locations here are separated by ',' followed by lots of spacing

In [11]:
df.details[0]

'About Internship:               We are seeking a HR Recruiter who will help us hire resources in technology domains such as Java, Node, React, Angular. You will work & collaborate with senior members to track & measure performance, and provide results for recruitment efforts.Skill(s) required: English Proficiency - Spoken & Written, Persuasion Skills                Roles and Responsibilities:               1. Handling the recruitment for latest & emerging technologies such as Node, Java-SpringBoot, 2. Streamlining & improvising company processes such as appraisals                Perks:               1. Internship Certificate2. 5 days a week'

In [12]:
df.details[5]

"About Internship:               Mba Finance students who are keen to learn about Capital markets, can do portfolio management of clients. Mba Finance students who are ready to put the hard work & have it in them what it takes to be an successful portfolio manager. Mba Finance students who are interested in capital markets with preferred location as Bangalore should only apply.                Roles and Responsibilities:               1. Developing and managing portfolio as per clients requirements.2. Track, analyse & optimize portfolios on a day to day basis.3. Devisiing strategies to get acquire fresh clients.4. Devising Financial models for maximum profits.5. Track & analyse market volatilities on a day to basis & create strategies to maximise returns on Clients investments with regards to market volatility.6. Developing the WOW factor in Clients mindspace by offering them enhanced services.7. Increase Brand's market share through online & offline tools.                Perks:        

In [13]:
df.details[100]

'Hello,Good opportunity to work with experienced professionals helping them developing their product. Gain a well startup experience.\xa0Opportunity will be to show your skills and talent. We are based in UK and India. \xa090% of the time you have work virtually and should be available to discuss on skype / facetime. \xa0After establishing the target audience for a website and identifying the type of content it will host, a web designer/developer will:write the programming code, either from scratch or by adapting existing website software and graphics packages to meet business requirementstest the website and identify any technical problemsupload the site onto a server and register it with different search engines.'

In [14]:
df.details[300]

'About Internship:               Lawyered is seeking an enthusiastic, creative, and dependable marketing intern to join our growingmarketing team. In this position, you will collaborate with our marketing and advertising teams ineach stage of marketing campaigns. Your work will include developing, growing, and expanding ourmarketing channels through social media outreach, blogging, and strategic planning.                Roles and Responsibilities:               ? Support the marketing team? Strategize end to end marketing campaigns using digital tools.? Monitor SEO and user engagement and suggest content optimization.? Assist in the creation of email campaigns, and social media content.? Seek and analyze competitor marketing material and digital content.? Create and maintain tracking reports of digital and traditional marketing efforts.? Develop new social media campaigns, considering current and planned promotional activities.? Monitor social channels and respond to feedback, question

What we see is that details section is pretty different for most of them but has a structure where it starts with 'About Intenrhsip' followed by 'Roles and Responsibilities' and has 'Perks' at the end. If there is a skills required section, it can be removed. Also, we will remove '\xa0' from all the columns replacing them with ''.

In [15]:
df = df.apply(lambda x: x.str.replace('\xa0',''))

In [16]:
df.details[100]

'Hello,Good opportunity to work with experienced professionals helping them developing their product. Gain a well startup experience.Opportunity will be to show your skills and talent. We are based in UK and India. 90% of the time you have work virtually and should be available to discuss on skype / facetime. After establishing the target audience for a website and identifying the type of content it will host, a web designer/developer will:write the programming code, either from scratch or by adapting existing website software and graphics packages to meet business requirementstest the website and identify any technical problemsupload the site onto a server and register it with different search engines.'

Looks good !

In [17]:
df.skills.value_counts()

Skills Required                  Analytical Skills                                                                                                                                                                                                                                     80
Skills Required                  No skills preferred                                                                                                                                                                                                                                   41
Skills Required                  Accounting                                                                                                                                                                                                                                            41
Skills Required                  Agreeableness                                                                                                            

Above, different skills have been separated by spacing and skills required is written in every entry which we will have to remove.

In [18]:
df['skills'] = df.skills.str.replace('Skills Required','')

In [19]:
df = pd.concat([df.iloc[:,1:].apply(lambda x: x.str.lower()),df.href], axis = 1)

In [20]:
df.tail()

,job_title,company_name,job_loc,details,category,compensation,start,end,skills,href
648,sales manager,rj health products private ltd,"mumbai, thane, ...",about internship: we are seeking...,sales manager,recurring,28th feb 2019,nov ---- nov -,salesforce ...,http://letsintern.com/internship/Sales-Manager...
649,asp.net c# developer,dataligence infotech pvt ltd,ahmedabad,about internship: we are looking...,software developer : c#,expenses covered,27th feb 2019,27th feb 2020,c# ...,http://letsintern.com/internship/Software-Deve...
650,marketing,gajra pai and zhu limited,"gurgaon, chennai",about internship: we are looking...,marketing assistant,recurring,1st may 2019,nov ---- nov -,analytical skills,http://letsintern.com/internship/Marketing-Ass...
651,marketing internship,concinnity agro2o pvt. ltd.,new delhi,about internship: agro2o® is new...,marketing professional,recurring,26th jan 2019,27th mar 2019,marketing ...,http://letsintern.com/internship/Marketing-Pro...
652,content writer,www.onlineflowersgift.com,"gurgaon, bangalore,...",about internship: we are looking...,content manager,recurring,21st aug 2018,30th aug 2020,analytical skills,http://letsintern.com/internship/Content-Manag...


Looks all good for the 2nd step

3. DATE INTO DATETIME OBJECTS

In [21]:
df.end.head()

0      28th aug 2019
1      30th aug 2019
2      29th apr 2020
3    nov  ---- nov -
4      20th jun 2019
Name: end, dtype: object

We must handle the 4th type of entry above which means that no end date has been specified for the internship

In [22]:
df.start = pd.to_datetime(df.start)

In [23]:
# errors = 'coerce' will leave a null type object where parsing can't happen
df.end = pd.to_datetime(df.end, errors = 'coerce')

In [24]:
df.end.head()

0   2019-08-28
1   2019-08-30
2   2020-04-29
3          NaT
4   2019-06-20
Name: end, dtype: datetime64[ns]

In [25]:
# number of null entries in end date column
df.isnull().sum().end

222

We will let these by NaT right now and decide what to do about it later

4. COMPENSATION

In [26]:
df.compensation.value_counts()

recurring                   348
expenses covered            126
unpaid                       95
one-time                     82
paid (monthly, variable)      2
Name: compensation, dtype: int64

We will club these into 2 catrgories : 

1. recurring = paid
2. expenses covered = paid
3. unpaid = unpaid
4. one-time = paid
5. paid (monthly, variable) = paid

These will be used in the knowledge based part of the recommendation system.

In [27]:
compensation_dict = {'recurring':'paid', 'one-time':'paid', 'paid (monthly, variable)':'paid', 
                     'expenses covered':'paid', 'unpaid':'unpaid'}

In [28]:
df.compensation = df.compensation.map(compensation_dict)

In [29]:
df.compensation.value_counts()

paid      558
unpaid     95
Name: compensation, dtype: int64

Perfect !!

5. SKILLS AND JOB LOCATION

In [30]:
# regex matches portion where there are 2 or more spaces and right before them is a no-space character
df.skills = df.skills.str.replace('(?<=\S)\s{2,}',',', regex = True)

In [31]:
# similar to above
df.job_loc= df.job_loc.str.replace('(?<=\S)\s{2,}','', regex = True)

6. SPLITTING DETAILS 

In [32]:
df.details.str.contains('about internship:').sum()

603

In [33]:
df.details.str.contains('roles and responsibilities').sum()

597

In [34]:
df.details.str.contains('skill').sum()

316

In [35]:
df.details.str.contains('perks').sum()

521

In [36]:
df.details = df.details.str.replace('about internship:','')

In [37]:
df.details = df.details.str.replace('roles and responsibilities:', '')

In [38]:
df.details = df.details.str.split('skill', expand = True)[0]

In [39]:
df.details = df.details.str.split('perks', expand = True)[0]

What we did above was that we replaced some keywords with spaces and then we split all the rows (for details) on 'skill' and took the portion of the split before 'skill' as we already have skills needed with us. Then we did the same thing for perk as we didn't need perks and the information before perks was good enough for us.

In [40]:
df.head()

,job_title,company_name,job_loc,details,category,compensation,start,end,skills,href
0,hr executive - recruitment,engenia technologies,gurgaon,we are seeking a hr recruiter w...,human resources recruiter,paid,2019-03-02,2019-08-28,hr practices,http://letsintern.com/internship/Human-Resourc...
1,telecalling & lead generation,abalone technologies pvt ltd,noida,selected intern's day-to-day re...,tele sales executive,paid,2019-02-17,2019-08-30,office administration,http://letsintern.com/internship/Tele-Sales-Ex...
2,digital marketing internship,brandstory digital marketing company,bangalore,are you looking for digital mar...,marketing professional,paid,2018-12-25,2020-04-29,digital marketing,http://letsintern.com/internship/Marketing-Pro...
3,recruitment of corporate bank back office post,bandhan pvt.ltd,"kathua,barasat,bardhaman,bongoan,habra",huge opportunity in corporate b...,accountant,paid,2019-03-12,NaT,analytical skills,http://letsintern.com/internship/Accountant-in...
4,software developer,trippyigloo,bangalore,we are looking for interns who ...,software developer : python,paid,2019-01-30,2019-06-20,"go(golang),java,mongodb,ngin...",http://letsintern.com/internship/Software-Deve...


In [41]:
# adding id column
df['id'] = range(1,df.shape[0]+ 1)
# changing the order
df = pd.concat([df['id'], df.iloc[:,:-1]], axis = 1)

In [42]:
# verifying nothing has gone wrong
df.isnull().sum()

id                0
job_title         0
company_name      0
job_loc           0
details           0
category          0
compensation      0
start             0
end             222
skills            0
href              0
dtype: int64

7. Removing duplicates

Now there are some duplicate entries that wont show up as duplicates, i.e, every column wouldn't be the same eg: start and end date might be different and even the details section might be a little different. This probably happens because of companies putting up internship openings at different times of the year and the previous outdated internship openings not being closed by letsintern.

Thus our definition for duplicates is that the job loc, category, company name and job title are the same.

In [43]:
# arranging internships by date so that new internships are at the top
df.sort_values(by ='start', ascending = False, inplace =True)

In [44]:
# take a look at the duplicates. For most, the dates are what are different
all_duplicates = df[df.duplicated(subset = ['job_title','company_name','job_loc','category'],keep=False)]
all_duplicates

,id,job_title,company_name,job_loc,details,category,compensation,start,end,skills,href
518,519,chief editor,holiday tripffee llp,anywhere in india,selected intern's day-to-day re...,content manager,unpaid,2019-04-14,2019-06-13,no skills preferred,http://letsintern.com/internship/Content-Manag...
282,283,chief editor,holiday tripffee llp,anywhere in india,selected intern's day-to-day re...,content manager,unpaid,2019-04-14,2019-06-13,agreeableness,http://letsintern.com/internship/Content-Manag...
116,117,finance,indian wealth management,ahmedabad,• applicants should be business...,finance professional,paid,2019-03-31,2019-10-30,"accounting,analytical skills",http://letsintern.com/internship/Finance-Profe...
220,221,business development,artsy nest,"hyderabad,mumbai","we are looking for young, motiv...",marketing manager,paid,2019-03-31,2019-06-29,"marketing,digital marketing,...",http://letsintern.com/internship/Marketing-Man...
558,559,business deveopment manager,3deometry innovations llp,pune,1. are available for full time ...,business development executive,paid,2019-03-31,2019-06-29,"marketing,electronics & semi...",http://letsintern.com/internship/Business-Deve...
95,96,business development,artsy nest,"hyderabad,mumbai","we are looking for young, motiv...",marketing manager,paid,2019-03-31,2019-06-29,"marketing,digital marketing,...",http://letsintern.com/internship/Marketing-Man...
176,177,digital marketing,sm consultants,hyderabad,we are seeking a digital market...,digital marketing manager,paid,2019-03-22,2019-06-21,"marketing,search engine mark...",http://letsintern.com/internship/Digital-Marke...
244,245,direct walk in interview for pvt bank back office,bandhan pvt.ltd,anywhere in india,staff selections going on for p...,accountant,paid,2019-03-13,NaT,no skills preferred,http://letsintern.com/internship/Accountant-in...
313,314,direct walk in interview for pvt bank back office,bandhan pvt.ltd,anywhere in india,staff selections going on for p...,accountant,paid,2019-03-13,NaT,no skills preferred,http://letsintern.com/internship/Accountant-in...
523,524,direct walk in interview for pvt bank back office,bandhan pvt.ltd,anywhere in india,staff selections going on for p...,accountant,unpaid,2019-03-13,NaT,no skills preferred,http://letsintern.com/internship/Accountant-in...


In [45]:
# we might also check the internships that were meant to start before March 2019
df[df.start < pd.to_datetime('03/01/2019')].shape

(315, 11)

Damn. 315  outdated internships. We could remove these to make our data more relevant to the dates but that won't matter as having an outdated internship won't affect the recommendation system making part. But we will still remove the duplicate internships.

In [46]:
df.drop_duplicates(subset = ['job_title','company_name','job_loc','category'],keep='first',inplace = True)

In [47]:
df.sort_values('id', inplace = True)

In [48]:
df.to_csv('../data/recomm_df.csv', sep = ',',index  = False)

This form of data is good enough for us to start working on the recommendatio system with. However if we wanted to analyze this dataset, we would need to change quite a few things.

1. SEPARATE SKILLS INTO DIFFERENT COLUMNS

In [49]:
def create_cats(col,df):
    '''
    returns a dataframe with dummy variables created for all values in col of df
    '''
    df_split = df[col].str.split(',', expand = True)
    # getting dummies for all the column values
    df_split_dummies = pd.get_dummies(df_split)
    # standardizing the column names
    df_split_dummies.columns = df_split_dummies.columns.str.replace('^[0-9][0-9]?_\s*',col+'_')
    # combining same columns together
    df_split = df_split_dummies.groupby(df_split_dummies.columns, axis = 1).sum()
    # combine with original df
    df = pd.concat([df, df_split], axis = 1)
    return df

In [50]:
df = create_cats('skills', df)

2. SPLIT SKILLS INTO DIFFERENT COLUMNS

In [51]:
df = create_cats('job_loc', df)

In [52]:
df.shape

(626, 327)

In [53]:
df.drop(['job_loc','skills'],axis= 1, inplace = True)

In [54]:
df.to_csv('../data/recomm_df_w_dummies.csv',index = False)